In [ ]:
import pandas as pd
from dspy.retrieve.qdrant_rm import QdrantRM
from qdrant_client import QdrantClient
import dspy
from typing import List, Dict, Tuple
import numpy as np
import csv
import os
import json

In [ ]:
reports = [
"""RADIOLOGY REPORT
Exam: Chest X-ray (2 views)
Date: XXXX, XXXX
Indication: Fever, cough
Findings: There is a focal area of consolidation in the right lower lobe. No pleural effusion or pneumothorax. The heart size is normal.
IMPRESSION: Right lower lobe pneumonia.
""",

"""RADIOLOGY REPORT
Exam: Chest radiograph
Date: XXXX, XXXX
Indication: Dyspnea
Findings: Bilateral interstitial opacities consistent with pulmonary edema. The cardiac silhouette is enlarged. No pneumothorax.
IMPRESSION: Cardiomegaly with pulmonary edema.
""",

"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: Chest pain
Findings: No acute cardiopulmonary abnormality. Lungs are clear. Heart size and mediastinal contours are within normal limits.
IMPRESSION: Normal chest X-ray.
""",

"""RADIOLOGY REPORT
Exam: Chest radiograph
Date: XXXX, XXXX
Indication: Trauma
Findings: Large left pneumothorax with partial lung collapse. No rib fractures visualized.
IMPRESSION: Left pneumothorax, likely tension pneumothorax given the degree of lung collapse.
""",

"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: Post-operative evaluation
Findings: Small bilateral pleural effusions, left greater than right. Mild pulmonary edema. Heart size is at the upper limits of normal.
IMPRESSION: Bilateral pleural effusions with mild pulmonary edema.
""",

"""RADIOLOGY REPORT
Exam: Portable chest X-ray
Date: XXXX, XXXX
Indication: Follow-up pneumonia
Findings: Persistent right lower lobe consolidation, improved from prior. No pneumothorax or pleural effusion.
IMPRESSION: Improving right lower lobe pneumonia.
""",

"""RADIOLOGY REPORT
Exam: Chest radiograph
Date: XXXX, XXXX
Indication: Shortness of breath
Findings: Diffuse bilateral airspace opacities consistent with pulmonary edema. The cardiac silhouette is markedly enlarged.
IMPRESSION: Severe cardiomegaly with pulmonary edema.
""",

"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: COPD exacerbation
Findings: Hyperinflated lung fields. No focal consolidation, pneumothorax, or pleural effusion. Heart size is normal.
IMPRESSION: Changes consistent with COPD. No acute cardiopulmonary process.
""",

"""RADIOLOGY REPORT
Exam: Chest radiograph
Date: XXXX, XXXX
Indication: Fever, productive cough
Findings: Patchy consolidation in the right upper lobe. Small right pleural effusion. Heart size is normal.
IMPRESSION: Right upper lobe pneumonia with parapneumonic effusion.
""",

"""RADIOLOGY REPORT
Exam: Portable chest X-ray
Date: XXXX, XXXX
Indication: Assess ET tube placement
Findings: ET tube is in proper position. Diffuse bilateral opacities consistent with ARDS. No pneumothorax.
IMPRESSION: ARDS pattern. Properly positioned ET tube.
""",

"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: Annual check-up
Findings: Clear lung fields. Normal heart size and mediastinal contours. No pleural effusion or pneumothorax.
IMPRESSION: Normal chest X-ray.
""",

"""RADIOLOGY REPORT
Exam: Chest radiograph
Date: XXXX, XXXX
Indication: Chest pain, shortness of breath
Findings: Large left pleural effusion with associated atelectasis. The heart is shifted to the right. No pneumothorax.
IMPRESSION: Large left pleural effusion with mediastinal shift.
""",

"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: Follow-up cardiomegaly
Findings: Persistent cardiomegaly with no change from prior studies. No pulmonary edema, pleural effusion, or pneumothorax.
IMPRESSION: Stable cardiomegaly without acute cardiopulmonary process.
""",

"""RADIOLOGY REPORT
Exam: Portable chest radiograph
Date: XXXX, XXXX
Indication: Evaluate lung collapse
Findings: Complete opacification of the left hemithorax with rightward mediastinal shift, consistent with total left lung collapse. Small left pneumothorax noted.
IMPRESSION: Left lung collapse with small pneumothorax.
""",

"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: Cough, fever
Findings: Patchy bilateral consolidations, more prominent in the lower lobes. No pleural effusion or pneumothorax. Heart size is normal.
IMPRESSION: Bilateral pneumonia, likely viral.
""",
"""RADIOLOGY REPORT
Exam: Chest X-ray (2 views)
Date: XXXX, XXXX
Indication: Shortness of breath
Findings: Bilateral perihilar opacities consistent with pulmonary edema. The cardiac silhouette is enlarged. No pneumothorax or pleural effusion.
IMPRESSION: Pulmonary edema with cardiomegaly.
""",
"""RADIOLOGY REPORT
Exam: Chest radiograph
Date: XXXX, XXXX
Indication: Fever, cough
Findings: Focal consolidation in the right lower lobe. No pleural effusion or pneumothorax. Heart size within normal limits.
IMPRESSION: Right lower lobe pneumonia.
""",
"""RADIOLOGY REPORT
Exam: Portable chest X-ray
Date: XXXX, XXXX
Indication: Post-operative evaluation
Findings: Small bilateral pleural effusions, left greater than right. Lungs are otherwise clear. Heart size is normal.
IMPRESSION: Bilateral pleural effusions, likely post-operative.
""",
"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: Trauma
Findings: Large left pneumothorax with partial lung collapse. No rib fractures visualized. Heart size and mediastinal contours are normal.
IMPRESSION: Left pneumothorax.
""",
"""RADIOLOGY REPORT
Exam: Chest radiograph
Date: XXXX, XXXX
Indication: Annual check-up
Findings: Clear lung fields. Normal heart size and mediastinal contours. No pleural effusion or pneumothorax.
IMPRESSION: Normal chest X-ray.
""",
"""RADIOLOGY REPORT
Exam: Chest X-ray (2 views)
Date: XXXX, XXXX
Indication: Chest pain, shortness of breath
Findings: Diffuse bilateral airspace opacities consistent with pulmonary edema. The cardiac silhouette is markedly enlarged. Small bilateral pleural effusions.
IMPRESSION: Severe cardiomegaly with pulmonary edema and pleural effusions.
""",
"""RADIOLOGY REPORT
Exam: Portable chest radiograph
Date: XXXX, XXXX
Indication: Follow-up pneumonia
Findings: Persistent right upper lobe consolidation, improved from prior. No pneumothorax or pleural effusion. Heart size is normal.
IMPRESSION: Improving right upper lobe pneumonia.
""",
"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: COPD exacerbation
Findings: Hyperinflated lung fields. No focal consolidation, pneumothorax, or pleural effusion. Heart size is normal.
IMPRESSION: Changes consistent with COPD. No acute cardiopulmonary process.
""",
"""RADIOLOGY REPORT
Exam: Chest radiograph
Date: XXXX, XXXX
Indication: Fever, productive cough
Findings: Patchy consolidations in both lower lobes. Small right pleural effusion. Heart size is normal.
IMPRESSION: Bilateral lower lobe pneumonia with small right pleural effusion.
""",
"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: Shortness of breath
Findings: Increased interstitial markings bilaterally, consistent with pulmonary edema. The cardiac silhouette is mildly enlarged. No pneumothorax or pleural effusion.
IMPRESSION: Mild pulmonary edema and cardiomegaly.
""",
"""RADIOLOGY REPORT
Exam: Portable chest radiograph
Date: XXXX, XXXX
Indication: ICU follow-up
Findings: Diffuse bilateral airspace opacities, worse than prior. Large right pleural effusion. Left pneumothorax with chest tube in place.
IMPRESSION: Worsening bilateral pneumonia, right pleural effusion, and left pneumothorax with chest tube.
""",
"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: Pre-operative evaluation
Findings: Clear lung fields. Normal heart size and mediastinal contours. No pleural effusion or pneumothorax.
IMPRESSION: Normal chest X-ray.
""",
"""RADIOLOGY REPORT
Exam: Chest radiograph
Date: XXXX, XXXX
Indication: Chest pain
Findings: The cardiac silhouette is enlarged. Lungs are clear. No pneumothorax or pleural effusion.
IMPRESSION: Cardiomegaly without acute pulmonary findings.
""",
"""RADIOLOGY REPORT
Exam: Chest X-ray (2 views)
Date: XXXX, XXXX
Indication: Cough, fever
Findings: Focal area of consolidation in the left lower lobe. No pleural effusion or pneumothorax. Heart size is normal.
IMPRESSION: Left lower lobe pneumonia.
""",
"""RADIOLOGY REPORT
Exam: Portable chest radiograph
Date: XXXX, XXXX
Indication: Dyspnea
Findings: Bilateral interstitial and alveolar opacities consistent with pulmonary edema. The cardiac silhouette is at the upper limits of normal. Small bilateral pleural effusions.
IMPRESSION: Pulmonary edema with small bilateral pleural effusions.
""",
"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: Follow-up lung nodule
Findings: No focal consolidation, pneumothorax, or pleural effusion. Heart size and mediastinal contours are within normal limits. Previously noted lung nodule is stable.
IMPRESSION: No acute cardiopulmonary process. Stable lung nodule.
""",
"""RADIOLOGY REPORT
Exam: Chest radiograph
Date: XXXX, XXXX
Indication: Trauma
Findings: Large right pneumothorax with partial lung collapse. Multiple rib fractures on the right side. Heart size and mediastinal contours are normal.
IMPRESSION: Right pneumothorax with associated rib fractures.
""",
"""RADIOLOGY REPORT
Exam: Chest X-ray (2 views)
Date: XXXX, XXXX
Indication: Shortness of breath, fever
Findings: Patchy bilateral consolidations, more prominent in the lower lobes. Small bilateral pleural effusions. The cardiac silhouette is mildly enlarged.
IMPRESSION: Bilateral pneumonia with small pleural effusions and mild cardiomegaly.
""",
"""RADIOLOGY REPORT
Exam: Portable chest radiograph
Date: XXXX, XXXX
Indication: Post-cardiac surgery follow-up
Findings: Stable cardiomegaly. Small left pleural effusion. No pneumothorax or pulmonary edema.
IMPRESSION: Cardiomegaly with small left pleural effusion, likely post-operative.
""",
"""RADIOLOGY REPORT
Exam: Chest X-ray
Date: XXXX, XXXX
Indication: Routine surveillance
Findings: Clear lung fields. Normal heart size and mediastinal contours. No pleural effusion or pneumothorax.
IMPRESSION: Normal chest X-ray.
"""
]

In [ ]:
ground_truth = [
["consolidation"],
["pulmonary edema", "cardiomegaly"],
[],
["pneumothorax"],
["pleural effusion", "pulmonary edema"],
["consolidation"],
["pulmonary edema", "cardiomegaly"],
[],
["consolidation", "pleural effusion"],
[],
[],
["pleural effusion"],
["cardiomegaly"],
["pneumothorax"],
["consolidation"],
["pulmonary edema", "cardiomegaly"],
["consolidation"],
["pleural effusion"],
["pneumothorax"],
[],
["pulmonary edema", "cardiomegaly", "pleural effusion"],
["consolidation"],
[],
["consolidation", "pleural effusion"],
["pulmonary edema", "cardiomegaly"],
["consolidation", "pleural effusion", "pneumothorax"],
[],
["cardiomegaly"],
["consolidation"],
["pulmonary edema", "pleural effusion"],
[],
["pneumothorax"],
["consolidation", "pleural effusion", "cardiomegaly"],
["cardiomegaly", "pleural effusion"],
[]
]
classes = ["pulmonary edema", "consolidation", "pleural effusion", "pneumothorax", "cardiomegaly"]

In [ ]:
len(reports)

In [ ]:
len(ground_truth)

In [ ]:
import logging

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [ ]:
def clean_json_string(json_str: str) -> str:
    # Remove the backticks and the "json" text
    return json_str.replace('```json\n', '').replace('\n```', '')

In [ ]:
class ClassifyText(dspy.Signature):
    """Classify the radiology into multiple labels from the given candidates. You should return the 
    extracted information as a single JSON string with a key for each candidate label and a value of
    1 if the report indicates the presence of the abnormality and 0 otherwise. There should be no 
    text or explanation, only the JSON. For example if there 
    were 3 candidates you could have the following output:

    {
        "label_1": 1,
        "label_2": 0,
        "label_3": 1
    }"""
    text = dspy.InputField()
    label_candidates = dspy.InputField(desc="List of candidate labels for the text")
    rad_labels = dspy.OutputField(desc="Dictionary of candidate labels, 1 or 0, for the text")

# class RAGMultiLabelClassifier(dspy.Module):
#     def __init__(self, num_candidates=3):
#         super().__init__()
#         self.retrieve = dspy.Retrieve(k=num_candidates)
#         self.classify = dspy.Predict(ClassifyText)
    
#     def forward(self, text):
#         retrieved_docs = ','.join(self.retrieve(text).passages)
#         classification_result = self.classify(text=text, label_candidates=retrieved_docs)
#         result = classification_result.rad_labels
#         result = clean_json_string(result)
#         # Parse the JSON string into a dictionary
#         return json.loads(result)

# Version below is for optimization and handles cases where the model outputs something other than JSON
# class RAGMultiLabelClassifier(dspy.Module):
#     def __init__(self, num_candidates=3):
#         super().__init__()
#         self.retrieve = dspy.Retrieve(k=num_candidates)
#         self.classify = dspy.Predict(ClassifyText)
    
#     def forward(self, text):
#         retrieved_docs = ','.join(self.retrieve(text).passages)
#         print(retrieved_docs)
#         classification_result = self.classify(text=text, label_candidates=retrieved_docs)
#         result = classification_result.rad_labels
#         result = clean_json_string(result)
        
#         logger.debug(f"Raw classification result: {result}")
        
#         try:
#             parsed_result = json.loads(result)
#         except json.JSONDecodeError:
#             # If JSON parsing fails, try to extract a dictionary-like structure
#             import re
#             dict_match = re.search(r'\{.*\}', result, re.DOTALL)
#             if dict_match:
#                 dict_str = dict_match.group(0)
#                 try:
#                     parsed_result = eval(dict_str)
#                 except:
#                     parsed_result = {"wrong": 1}  # Fallback to hard-coded wrong output
#             else:
#                 parsed_result = {"wrong": 1}  # Fallback to hard-coded wrong output
        
#         # Ensure the output is a dictionary
#         if not isinstance(parsed_result, dict):
#             parsed_result = {"wrong": 1}  # Fallback to hard-coded wrong output
        
#         logger.debug(f"Final parsed result: {parsed_result}")
#         return parsed_result

class RAGMultiLabelClassifier(dspy.Module):
    def __init__(self, custom_retriever, num_candidates=3):
        super().__init__()
        self.retrieve = custom_retriever
        self.classify = dspy.Predict(ClassifyText)
        self.num_candidates = num_candidates
    
    def forward(self, text):
        retrieved_docs = self.retrieve(text, k=self.num_candidates)
        retrieved_labels = ','.join([doc['long_text'] for doc in retrieved_docs])
        print(retrieved_labels)
        
        classification_result = self.classify(text=text, label_candidates=retrieved_labels)
        result = classification_result.rad_labels
        result = clean_json_string(result)
        
        logger.debug(f"Raw classification result: {result}")
        
        try:
            parsed_result = json.loads(result)
        except json.JSONDecodeError:
            # If JSON parsing fails, try to extract a dictionary-like structure
            import re
            dict_match = re.search(r'\{.*\}', result, re.DOTALL)
            if dict_match:
                dict_str = dict_match.group(0)
                try:
                    parsed_result = eval(dict_str)
                except:
                    parsed_result = {"wrong": 1}  # Fallback to hard-coded wrong output
            else:
                parsed_result = {"wrong": 1}  # Fallback to hard-coded wrong output
        
        # Ensure the output is a dictionary
        if not isinstance(parsed_result, dict):
            parsed_result = {"wrong": 1}  # Fallback to hard-coded wrong output
        
        logger.debug(f"Final parsed result: {parsed_result}")
        return parsed_result
    
def build_retriever_client(labels: List[str], collection_name: str, k: int, vectorizer: str = None) -> QdrantRM:
    client = QdrantClient(":memory:")
    ids = list(range(len(labels)))
    
    if vectorizer:
        client.set_model(vectorizer)
        
    client.add(
        collection_name=collection_name,
        documents=labels,
        ids=ids
    )
    return QdrantRM(collection_name, client, k=k)

In [ ]:
def parse_ollama_output(output_str: str, clean_values: bool = True) -> List[str]:
    if clean_values:
        # Remove the backticks and the "json" text
        output_str = clean_json_string(output_str)
    output_dict = json.loads(output_str)
    predicted_classes = [key for key, value in output_dict.items() if value == 1]
    return predicted_classes

In [ ]:
vectorizer = "intfloat/multilingual-e5-large"
# vectorizer = "sentence-transformers/all-MiniLM-L6-v2"
# vectorizer = "nomic-ai/nomic-embed-text-v1.5-Q"
ollama_model_name = 'gemma2'

In [ ]:
retriever_model = build_retriever_client(labels=classes, 
                                         collection_name="rad", 
                                         k=3, 
                                         vectorizer=vectorizer)

In [ ]:
reports[1]

In [ ]:
for report in reports:
    output = retriever_model(report, k=3)
    print(output)

In [ ]:
# ollama_model = dspy.OllamaLocal(
#     model=ollama_model_name, 
#     model_type='text',
#     max_tokens=512,
#     temperature=0,
#     top_p=1,
#     frequency_penalty=0,
#     top_k=5,
#     format='json'
# )

#### We may need to create a custom Ollama model with a copy method

In [ ]:
# class CustomOllamaLocal(dspy.OllamaLocal):
#     def copy(self, **kwargs):
#         new_kwargs = self.__dict__.copy()
#         new_kwargs.update(kwargs)
#         return CustomOllamaLocal(**new_kwargs)

class CustomOllamaLocal(dspy.OllamaLocal):
    def __init__(self, model, **kwargs):
        logger.debug(f"Initializing CustomOllamaLocal with model: {model}")
        self.model = model  # Explicitly set the model attribute
        super().__init__(model=model, **kwargs)
        
    def copy(self, **kwargs):
        logger.debug(f"Copying CustomOllamaLocal with kwargs: {kwargs}")
        new_kwargs = self.__dict__.copy()
        new_kwargs.update(kwargs)
        return CustomOllamaLocal(**new_kwargs)
    
    def basic_request(self, prompt, **kwargs):
        logger.debug(f"Making basic request with model: {self.model}")
        return super().basic_request(prompt, **kwargs)

In [ ]:
ollama_model = CustomOllamaLocal(
    model=ollama_model_name, 
    model_type='text',
    max_tokens=512,
    temperature=0,
    top_p=1,
    frequency_penalty=0,
    top_k=5,
    format='json'
)

In [ ]:
logger.debug(f"Created ollama_model with model: {ollama_model.model_name}")

In [ ]:
dspy.settings.configure(lm=ollama_model, rm=retriever_model)
# classifier = RAGMultiLabelClassifier(num_candidates=3)

classifier = RAGMultiLabelClassifier(custom_retriever=retriever_model, num_candidates=3)

In [ ]:
for report in reports:
    output = classifier.retrieve(report, k=3)
    print(output)

## DSPy Optimizer

In [ ]:
def accuracy(pred, gold):
    pred_set = set(pred)
    gold_set = set(gold)
    return int(pred_set == gold_set)

# metric = dspy.Metric(accuracy)

In [ ]:
# def evaluate_model(model, dataset):
#     total_accuracy = 0
#     for example in dataset:
#         result_str = model(text=example['text'])
#         predicted_classes = parse_ollama_output(result_str)
#         total_accuracy += accuracy(predicted_classes, example['ground_truth'])
#     return total_accuracy / len(dataset)

def evaluate_model(model, dataset):
    total_accuracy = 0
    for example in dataset:
        result = model(text=example['text'])
        print(result)
        if isinstance(result, str):
            predicted_classes = parse_ollama_output(result)
        else:
            predicted_classes = [k for k, v in result.items() if v == 1]
        total_accuracy += accuracy(predicted_classes, example['ground_truth'])
    return total_accuracy / len(dataset)

In [ ]:
dataset = [{'text': report, 'ground_truth': label} for report, label in zip(reports, ground_truth)]

# dataset = [
#     {'text': reports[0], 'ground_truth': ground_truth[0]},
#     {'text': reports[1], 'ground_truth': ground_truth[1]},
#     # Add more examples as needed
# ]

## Create dspy `Example` dataset

In [ ]:
data = [dspy.Example(ground_truth=label, text=report).with_inputs("text") for report, label in zip(reports, ground_truth)]


In [ ]:
trainset = data[:20]
devset = data[20:30]
testset = data[30:]

## Test uncompiled inference

## TODO 
- Look into why the retriever model is always pulling in the same classes
- Increase the top k to 5

In [ ]:
# Evaluate before compilation
print("Accuracy before compilation:", evaluate_model(classifier, data))

In [ ]:
reports[1]

In [ ]:
ground_truth[1]

In [ ]:
classifier(reports[1])

In [ ]:
# parse_ollama_output(classifier(reports[1]))

## Create metric

In [ ]:
# def metric(gold, pred, trace=None):
#     predicted_classes = parse_ollama_output(pred)
#     acc = accuracy(predicted_classes, gold.ground_truth)

#     return acc

# def metric(gold, pred, trace=None):
#     if isinstance(pred, str):
#         predicted_classes = parse_ollama_output(pred)
#     else:
#         predicted_classes = [k for k, v in pred.items() if v == 1]
#     acc = accuracy(predicted_classes, gold.ground_truth)
#     return acc

def metric(gold, pred, trace=None):
    if isinstance(pred, str):
        try:
            pred = json.loads(pred)
        except json.JSONDecodeError:
            pred = {"wrong": 1}
    elif not isinstance(pred, dict):
        pred = {"wrong": 1}
    
    # Check if we got the fallback output
    if pred == {"wrong": 1}:
        return 0.0  # Ensure zero score for fallback output
    
    predicted_classes = [k for k, v in pred.items() if v == 1]
    acc = accuracy(predicted_classes, gold.ground_truth)
    return acc

### Bootstrap Few-Shot optimizer

In [ ]:
from dspy.teleprompt import BootstrapFewShot

In [ ]:
teleprompter = BootstrapFewShot(metric=metric, max_labeled_demos=10, max_rounds=4)

In [ ]:
compiled_rag = teleprompter.compile(classifier, trainset=trainset)

In [ ]:
# Evaluate after compilation
print("Accuracy after compilation:", evaluate_model(compiled_rag, dataset))

## Bootstrap few shot with random search

In [ ]:
def custom_merge_dicts(d1, d2):
    if isinstance(d1, dspy.Example):
        merged = d1._store.copy()
    else:
        merged = d1.copy()
    
    if isinstance(d2, str):
        merged['prediction'] = d2
    elif isinstance(d2, dict):
        for k, v in d2.items():
            if k in merged:
                merged[f"pred_{k}"] = v
            else:
                merged[k] = v
    return merged

In [ ]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

In [ ]:
teleprompter = BootstrapFewShotWithRandomSearch(metric=metric, 
                                                max_bootstrapped_demos=4,
                                                max_labeled_demos=4,
                                                max_rounds=3,
                                                num_candidate_programs=2,
                                                num_threads=2
                                               )

In [ ]:
import dspy.evaluate.evaluate as dspy_evaluate

# Save the original function
original_merge_dicts = dspy_evaluate.merge_dicts

# Replace with your custom function
dspy_evaluate.merge_dicts = custom_merge_dicts

# Now run your compilation
compiled_rag_2 = teleprompter.compile(classifier, trainset=trainset)

# After you're done, you can restore the original function if needed
# dspy_evaluate.merge_dicts = original_merge_dicts

In [ ]:
# Evaluate after compilation
print("Accuracy after compilation:", evaluate_model(compiled_rag_2, dataset))

## Bayesian optimizer

Interesting issue here is that we need the output to be JSON, but by allowing an LLM to rewrite our prompt we lose more control over that component. Therefore we need stronger enforcement of the output type. We could also use a fallback like a known/predictable wrong output when the output type is not JSON.

In [ ]:
from dspy.teleprompt import BayesianSignatureOptimizer

In [ ]:
model_name = 'llama3'

llm_prompter = CustomOllamaLocal(
    model=ollama_model_name, 
    model_type='chat',
    max_tokens=1024,
    temperature=0,
    top_p=1,
    frequency_penalty=0,
    top_k=5
)

In [ ]:
teleprompter = BayesianSignatureOptimizer(task_model=dspy.settings.lm,
                                          metric=metric,
                                          prompt_model=llm_prompter,
                                          n=5,
                                          verbose=False
                                         )

In [ ]:
kwargs = dict(num_threads=1, display_progress=True, display_table=0)

In [ ]:
compiled_rag_3 = teleprompter.compile(classifier, devset=devset,
                                      optuna_trials_num=3,
                                      max_bootstrapped_demos=4,
                                      max_labeled_demos=4,
                                      eval_kwargs=kwargs
                                     )

In [ ]:
print("Accuracy after compilation:", evaluate_model(compiled_rag_3, dataset))